# Setup spark

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell spark.executor.cores=2"

from pyspark.sql import SparkSession
from pyspark.sql.functions import flatten
from pyspark.sql.types import (StructType, StructField, StringType, 
                                FloatType, DateType, IntegerType, ArrayType)
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("BDA assignment") \
        .getOrCreate()

# Imports

In [2]:
from typing import NamedTuple, Final, List
#from lxml import etree
import xml.etree.ElementTree as ET
from itertools import islice, chain, combinations
import argparse
import traceback
import bleach
import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import pr
import string
import random
import hashlib
import pandas as pd
import numpy as np
import hashlib
import json
import sys
from itertools import islice

# Constants

In [3]:
SHINGLE_SIZE: Final = 5
SAMPLES: Final = 1000

In [4]:
class comment_tuple(NamedTuple):
    id: int
    #owner_id: int
    post_type: int
    score: int
    text: str

class shingle_set(NamedTuple):
    id: int
    shingles: frozenset[tuple]

class similarity(NamedTuple):
    id_set1: int
    id_set2: int
    similarity: float

# Read and clean XML

In [5]:
def set_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Body", StringType(), True))
    
    return StructType(schema_list)

def parse_post(rdd):
    results = []
    root = ET.fromstring(rdd[0])

    for elem in root.findall('row'):
        rec = []
        #print("Found row")
        assert elem.text is None, "The row wasn't empty"
        rec.append(int(elem.attrib["Id"]))
        #int(elem.attrib["OwnerUserId"]),
        #rec.append(int(elem.attrib["PostTypeId"])),
        #rec.append(int(elem.attrib["Score"])),
        rec.append(bleach.clean(elem.attrib["Body"], strip=True))
        #rec.append(elem.attrib["Body"])

        #elem.clear()
        #while elem.getprevious() is not None:
        #    del elem.getparent()[0]
        results.append(rec)

    return results

In [6]:
filename = "crypto_posts.bakxml"
chunksize = 1024

file_rdd = spark.read.text(filename, wholetext=True).rdd
dataset = file_rdd.flatMap(parse_post)

In [7]:
dataset.persist()
schema = set_schema()
df_ds = dataset.toDF(schema)
df_ds.show()

#df_posts = records_rdd.toDF(schema)
#coll = records_rdd.collect()

+---+--------------------+
| Id|                Body|
+---+--------------------+
|  3|Why do we use a p...|
|  6|The initial and f...|
|  7|When initiating a...|
|  8|As we know, <a hr...|
|  9|Since most crypto...|
| 12|Wikipedia has a r...|
| 13|Here is the relev...|
| 14|RSA is not design...|
| 15|The solution to t...|
| 16|It seems like the...|
| 17|<a href="http://w...|
| 18|I understand the ...|
| 19|There is a <a hre...|
| 20|AES has several d...|
| 21|Yes, you need to ...|
| 22|Usually, when the...|
| 23|For practical pur...|
| 24|Using a cryptogra...|
| 25|The general justi...|
| 26|the securty of 1-...|
+---+--------------------+
only showing top 20 rows



# Shingling

In [8]:
class shingler:
    """
    Class that contain a tokenizer and stopwords to make shingling easier.
    """
    def __init__(self):
        self.stopwords = set(stopwords.words('english'))

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        return True

    def tokenize(self, text: str) -> List[str]:
        text = text.translate(str.maketrans('', '', string.punctuation))
        # text_nop = text.split()
        text_nop = word_tokenize(text)
        filtered_words = []

        for word in text_nop:
            if word not in self.stopwords:
                filtered_words.append(word.lower())
        
        return filtered_words

    #def create_shingle(self, input_comment: comment_tuple, shingle_size: int) -> frozenset[tuple]:
    #    tokens = self.tokenize(input_comment.text)
    #    comment_length = len(tokens)
    #    shingles =  frozenset(tuple(tokens[i:(i + shingle_size)]) for i in range(comment_length - shingle_size + 1))
    
    def create_shingle(self, post: str, shingle_size: int) -> list[list]:
        tokens = self.tokenize(post)
        comment_length = len(tokens)
        shingle_set = frozenset(tuple(tokens[i:(i + shingle_size - 1)]) for i in range(comment_length - shingle_size))
        shingle_list = list(shingle_set)
        #for elem in shingle_list:
        #    elem.sort()
        shingle_list.sort()
        return list(shingle_set)
        

def shingle_map(row):
    ds_shingler = shingler()
    return (row[0], ds_shingler.create_shingle(row[1], SHINGLE_SIZE)
)

def set_shingle_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Shingles", ArrayType(ArrayType(StringType()), True)))
    return StructType(schema_list)

In [10]:
schema = set_shingle_schema()
shingle_rdd = dataset.map(shingle_map).sortByKey()

for elem in shingle_rdd.take(1):
    print(elem)
#df_shingle = shingle_rdd.toDF(schema)
#df_shingle.show()

(3, [('step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes', 'algorithma', 'one'), ('why', 'use', 'permutation', 'table'), ('permutation', 'table', 'first', 'step'), ('table', 'first', 'step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes'), ('use', 'permutation', 'table', 'first'), ('first', 'step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes', 'algorithma')])


# MinHashing

In [11]:
# Transform posts to characteristic matrix
# Make feature set matrix
# Minhash
# Make Minhash Matrix
# LSH
import random
SIGNATURE_SIZE: Final = 114
HASH_PRIME: Final = (1 << 31) - 1
MAX_HASH: Final = (1 << 32)
    #sys.maxsize
HASH_RANGE: Final = (1<< 32)
    #sys.maxsize
SEED: Final = 193120

generator = np.random.RandomState(SEED)
salts = [generator.randint(1, MAX_HASH) for _ in range(SIGNATURE_SIZE)]
#permutations = [generator.randint(1, HASH_PRIME) for _ in range(SIGNATURE_SIZE)]

permutations = [[generator.randint(1, MAX_HASH) for _ in range(SIGNATURE_SIZE)],
                [generator.randint(0, MAX_HASH) for _ in range(SIGNATURE_SIZE)]]

print(salts)
print(permutations)
    
def min_hasher(row):
    sig = np.full((SIGNATURE_SIZE), MAX_HASH)
    for shingle in row[1]:
        for i in range(SIGNATURE_SIZE):
            a = permutations[0][i]
            b = permutations[1][i]
            hash_val = (a * hash(shingle) + b) % HASH_PRIME
            sig[i] = min(hash_val, sig[i])
    return (row[0], sig)

[2098660572, 74013022, 2675907171, 2481944613, 2289028529, 846582864, 1024003631, 3457496236, 958218556, 2872479854, 1385329197, 2720560315, 2596604670, 4118717270, 3831528778, 4184700433, 130382780, 4221132295, 2492231677, 1349853675, 2674011478, 4230155413, 522369175, 1349933754, 2597169981, 1045439184, 3199517916, 3020468163, 618450142, 3282454786, 4061764145, 3477766529, 3055070885, 204747729, 834378094, 4046990333, 2141660100, 2118590568, 3537961224, 168082228, 652073352, 1284401985, 2826423363, 1325707607, 3731067674, 723716237, 3333080471, 2787841228, 2113795590, 3230829213, 1843969928, 746019385, 2309594694, 2229839123, 489336389, 779150137, 1446103079, 889348879, 3745936412, 4281307413, 2173665908, 1717438567, 23839969, 2145022294, 1052302893, 1118678081, 1295195691, 3851408124, 3928788718, 2370966349, 561561790, 1051546482, 4224882136, 4206633849, 1329396735, 1801554849, 1138195103, 3123692201, 1122989246, 1270364612, 1593839649, 1463959047, 4229718824, 1709163888, 33163415, 

In [14]:
a = permutations[0][1]
b = permutations[1][1]

hash_val = (a * hash(tuple(["text", "word", "shingle", "advanced"]))+ b) % HASH_PRIME
print(hash_val)

hash_val = (a * hash(tuple(["text", "word", "shingle", "advanced"]))+ b) % HASH_PRIME
print(hash_val)

1762495466
1762495466


In [13]:
hash_rdd = shingle_rdd.map(min_hasher)

for elem in hash_rdd.take(3):
    print(elem)

(3, array([ 143118367,  974157036,  120529325,  233764212,  131981857,
        144173947,  102476572,  272330361,  325355145,  225106668,
        252430667,  119208473,  365627218,  274328953,  372891038,
        288125122,  105170397,   62620978,   68451392,  931263728,
         93321412,  498570867,  332666967,  105773097,   80245194,
        525535131,  428950765,   83103158,  212820571,  184943455,
        521919095,  284019571,  108475583,   80821552,  493994541,
         21562223,  686063402,   38664631,  316661511,  257903120,
        503435598,  317858444,  807428228,  392298432,  148846683,
        302211885,  643180557,   92896604,   40078093,    7972648,
        942859009,  417836429,  758783383,    8911154, 1035650124,
       1315691549,   12393071,   67626474,  397792855,  437018782,
        671242795,  971439139,   54582455,  595988434,  736171633,
         46422463,  757983522,  373394180,  587163066,   11155730,
        488219793,  108134471,  308860898,   83213833,  11

# LSH

In [15]:
BANDS: Final = 19
ROWS: Final = 6
THRESHOLD: Final = (1/BANDS) ** (1/ROWS)
print(f"Bands: {BANDS}, rows {ROWS}, threshold {THRESHOLD}")

def hash_func(row):
    sum = 0
    for e in row[1][0]:
        sum += e
    return (row[0], (int.from_bytes(hashlib.md5(str(sum).encode()).digest()[:4], byteorder="big"), row[1][1]))

Bands: 19, rows 6, threshold 0.612173322502962


In [16]:
# returns (doc, band, hash)
hash_band_rdd = hash_rdd.flatMap(lambda x: [[(x[0], i % BANDS), hash] for i, hash in enumerate(x[1])]).groupByKey().cache()

for elem in hash_band_rdd.take(5):
    print(elem)

((36, 18), <pyspark.resultiterable.ResultIterable object at 0x7f045f819e20>)
((53, 1), <pyspark.resultiterable.ResultIterable object at 0x7f045f819be0>)
((84, 4), <pyspark.resultiterable.ResultIterable object at 0x7f04cc3bfc40>)
((92, 16), <pyspark.resultiterable.ResultIterable object at 0x7f045f819400>)
((152, 12), <pyspark.resultiterable.ResultIterable object at 0x7f04cc3bf610>)


In [17]:
hash_bands_grouped_rdd = hash_band_rdd.map(lambda x: [x[0][1], (x[1], x[0][0])])

In [18]:
band_hashed = hash_bands_grouped_rdd.map(hash_func).map(lambda x: [(x[0], x[1][0]), x[1][1]]).groupByKey().filter(lambda x: (len(x[1]) > 1 and len(x[1]) < 50))

for elem in band_hashed.take(10):
    print(elem[0])
    for b in elem[1]:
        print(b)

(3, 568847360)
1244
1243
(7, 2495778086)
9801
9800
(17, 1264928584)
9801
9800
(0, 1693192036)
9848
9849
(3, 791037236)
10360
10361
(17, 414489296)
10621
10622
(5, 3048683361)
10892
10893
(2, 1823376585)
11093
11094
(10, 763011216)
12010
12011
(5, 1118652538)
12515
12516


In [19]:
candidates = band_hashed.map(lambda x: (tuple(x[1]), 1)).reduceByKey(lambda a, b: ((float(a + b) / 10.0))).cache()

for elem in candidates.take(100):
    print(elem)

((1244, 1243), 0.11111111112000001)
((9801, 9800), 0.2)
((9848, 9849), 0.11111111112000001)
((10360, 10361), 0.11112)
((10621, 10622), 0.11111111199999998)
((10892, 10893), 0.11111200000000002)
((11093, 11094), 0.11111120000000001)
((12010, 12011), 0.11111200000000002)
((12515, 12516), 0.11111112000000001)
((12516, 12515), 0.11111111199999998)
((59977, 50805), 0.11111112000000001)
((3410, 3409), 0.11111112000000001)
((9841, 9840), 0.11111111112000001)
((9849, 9848), 0.11111200000000002)
((10897, 10896), 0.11111111199999998)
((10943, 10942), 0.11120000000000001)
((22622, 22641), 0.11111111119999999)
((26874, 26234), 0.11111200000000002)
((32382, 31090), 0.11111112000000001)
((50805, 59977), 0.11111111199999998)
((60994, 21126), 1)
((64096, 64053), 0.11112)
((72388, 72389), 0.11111111199999998)
((72972, 29007), 1)
((84228, 84227), 0.11111120000000001)
((67066, 33475), 1)
((3409, 3410), 0.11111111199999998)
((10894, 3543), 0.11111111111111201)
((12013, 12012), 0.11111111199999998)
((8869,

In [20]:
cand = candidates.collect()
shingle_dict = shingle_rdd.collectAsMap()

In [21]:
def calc_jaccard(list1, list2):
    print(list1)
    print(list2)
    return len(set(list1).intersection(list2)) / len(set(list1).union(list2))

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print(take(1,shingle_dict.keys()))
print(take(1,shingle_dict.items()))

[3]
[(3, [('step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes', 'algorithma', 'one'), ('why', 'use', 'permutation', 'table'), ('permutation', 'table', 'first', 'step'), ('table', 'first', 'step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes'), ('use', 'permutation', 'table', 'first'), ('first', 'step', 'hrefhttpenwikipediaorgwikidataencryptionstandarddes', 'algorithma')])]


In [22]:
#candidate_sim = band_hashed.map(lambda x: [x[0], calc_jaccard(shingle_rdd.lookup(x[0][0]), shingle_rdd.lookup(x[0][1]))])

#for elem in candidate_sim.take(10):
#    print(elem)

for elem in cand:
    print(calc_jaccard(shingle_dict.get(elem[0][0]), shingle_dict.get(elem[0][1])))

[('white', 'box', 'analysis', 'means'), ('means', 'extracting', 'secret', 'value'), ('cryptography', 'white', 'box', 'analysis'), ('analysis', 'means', 'extracting', 'secret'), ('secret', 'value', 'implementation', 'function'), ('in', 'cryptography', 'white', 'box'), ('extracting', 'secret', 'value', 'implementation'), ('box', 'analysis', 'means', 'extracting')]
[('white', 'box', 'analysis', 'means'), ('means', 'extracting', 'secret', 'value'), ('cryptography', 'white', 'box', 'analysis'), ('analysis', 'means', 'extracting', 'secret'), ('secret', 'value', 'implementation', 'function'), ('in', 'cryptography', 'white', 'box'), ('extracting', 'secret', 'value', 'implementation'), ('box', 'analysis', 'means', 'extracting')]
1.0
[('hypothesis', 'given', 'set', 'data'), ('statistical', 'testing', 'used', 'estimate'), ('cryptanalysis', 'statistical', 'testing', 'commonly'), ('used', 'estimate', 'likelihood', 'hypothesis'), ('message', 'among', 'several', 'incorrect'), ('commonly', 'used', 'de

In [33]:
#for elem in hash_bands_grouped_rdd.take(2):
#    print(elem)
    #for h in elem[1]:
    #    print(h)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 15) (192.168.1.231 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/nix/store/4hn

In [15]:
bands_rdd = hash_band_rdd.map(lambda x: [(x[0][1], x[1]), x[0][0]]) 

for elem in bands_rdd.take(5):
    print(elem)

[(0, 67594442), 2]
[(1, 16439677), 2]
[(2, 45145227), 2]
[(3, 4436395), 2]
[(4, 8985140), 2]


In [16]:
vector_bucket_rdd = bands_rdd.map(lambda x: [x[1], x[0][1]]).distinct()

for elem in vector_bucket_rdd.take(1):
    print(elem)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (192.168.1.231 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/nix/store/4hn9f95hijjfhk93hx3wf87p38l5kbaq-python3.9-pyspark-3.1.1/lib/python3.9/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/nix/store/4hn9f9

# Exit Spark

In [ ]:
#spark.stop()